In [4]:
# change timezone to vietnam
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Ho_Chi_Minh /etc/localtime
!date

Tue Feb 11 01:40:59 PM +07 2025


In [ ]:
!pip install skpy
!pip install schedule
!pip install gspread oauth2client
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [6]:
USERNAME = 'tech07.qtdata@gmail.com'
PASSWORD = 'Tech@bot1234'

In [7]:
SKYPE_GROUP_ID = '19:7708c18fc9294263ae3c907a0b555226@thread.skype'
GOOGLE_SHEET_URL ='https://docs.google.com/spreadsheets/d/1CJhSkKmxfFFFxWGVEb8xxyvzWAPbl6Z9V99dK-iSiwg/edit?usp=sharing'

In [9]:
from skpy import Skype, SkypeMsg, SkypeSingleChat, SkypeGroupChat
from datetime import datetime, timedelta
import gspread
import pandas as pd
from google.auth import default
import pytz

In [10]:
def convert_time(time):
    return time + timedelta(hours=7) #Convert to GMT+7

In [11]:
# Fix <e_m></e_m>
import re

def clean_message(content):
    return re.sub(r"<[^>]+>", "", content)

In [18]:
def update_spreadsheet(df, spreadsheet_url=None):
        if spreadsheet_url is None:
            spreadsheet_url = GOOGLE_SHEET_URL

        gc = gspread.service_account(filename=fr"account.json")

        google_sheet = gc.open_by_url(spreadsheet_url)
        # print('sheetname',sheetname)
        sh = google_sheet.get_worksheet(0)


        str_list = list(filter(None, sh.col_values(1)))
        index_first_empty_row = len(str_list)+1
        # print('index_first_empty_row',index_first_empty_row)
        # if spreadsheet is empty, which mean first empty row's index = 1
        if index_first_empty_row == 1: #logic here
            # update data along with header
            sh.update(values = [df.columns.values.tolist()] + df.values.tolist())
        else:
            end_index = index_first_empty_row+df.shape[0]-1
            print('end_index',end_index)
            # only insert new row
            sh.update( range_name=f'A{index_first_empty_row}:D{end_index}', values=df.values.tolist())

In [19]:
group_topic = ""
print('Login success')
cur_date = datetime.today()
def get_group_message( group_id, num_day=2, sorted=False, update=False):
        """
        Get messages in the specified group within the last 'num_day' days.
        If 'sorted' is True, the messages will be sorted in ascending order of datetime.
        If 'update' is True, the data will be updated in Google Sheet.
        """

        sk = Skype(USERNAME, PASSWORD)
        if group_id is None:
            group_id = SKYPE_GROUP_ID

        channel = sk.chats[group_id]
        group_topic = channel.topic
        print('Group topic: ', group_topic)

        date_temp = cur_date
        date_previous = (date_temp - timedelta(days=num_day)).replace(hour=0, minute=0, second=0)
        list_message = []
        #get all msg within num_day
        while date_temp >= date_previous:
            print('date temp', date_temp)
            msgs = channel.getMsgs()
            if msgs is None or len(msgs) == 0:
                break
            list_message += msgs

            date_temp = convert_time(list_message[-1].time)
            print('Date_temp after conver', date_temp)

        df_data = pd.DataFrame(columns=['USERID', 'DATETIME', 'NAME', 'CONTENT'])
        for idx, message in enumerate(list_message):
            message_time = convert_time(message.time)
            print('msg content: ', message.content, ' date: ', message.time)
            if (message_time >= date_previous) and (message_time < date_previous + timedelta(days=num_day+1)):
                df_data.loc[idx, 'USERID'] = message.userId
                df_data.loc[idx, 'DATETIME'] = convert_time(message.time).strftime('%Y-%m-%d %H:%M:%S')
                df_data.loc[idx, 'NAME'] = message.user.name
                df_data.loc[idx, 'CONTENT'] = message.content
            else:
              print("time not valid")

        if sorted is True:
            df_data = df_data.sort_values('DATETIME')

        # Convert all columns' dtypes to string
        df_data['DATETIME'] = df_data['DATETIME'].astype('string')
        df_data['NAME'] = df_data['NAME'].astype('string')
        df_data['CONTENT'] = df_data['CONTENT'].astype('string')
        df_data['USERID'] = df_data['USERID'].astype('string')
        df_data['CONTENT'] = df_data['CONTENT'].apply(clean_message)

        # Filter out rows that are not messages created by users
        df_data = df_data[~df_data['CONTENT'].str.startswith('<')]

        if update is True:
            update_spreadsheet(df= df_data )

        return df_data

Login success


In [14]:
# def list_chat_ids():
#     chats = sk.chats.recent()
#     for chat_id, chat in chats.items():
#         if isinstance(chat, SkypeSingleChat):
#             print(f"chat Id: {chat_id}, chat with: {chat.userId} ")
#         elif isinstance(chat, SkypeGroupChat):
#             print(f"chat Id: {chat_id}, chat with: {chat.topic} ")
# list_chat_ids()

In [21]:
df = get_group_message(group_id=SKYPE_GROUP_ID, num_day=1, sorted=True, update=True)
df

Group topic:  AU-GlobalGroup: NỆN*iHugeNewRev*iSimpleAct*Top-iUp
date temp 2025-02-11 13:44:37.106717
Date_temp after conver 2025-02-10 13:26:44.109000
date temp 2025-02-10 13:26:44.109000
Date_temp after conver 2025-02-06 11:05:14.056000
msg content:  Nguyen_Ky_Duong - checkin -11/2/2025 -13h30-16h  date:  2025-02-11 06:35:48.661000
msg content:  Nguyen_Thi_ Bich_Ngoc - checkin - 11/2/2025 - 13h30-16h  date:  2025-02-11 06:31:55.267000
msg content:  Chau_Binh_Nguyen - checkin - 11/2/2025 - 13h30-16h  date:  2025-02-11 06:30:11.007000
msg content:  <addmember><eventtime>1739254556266</eventtime><initiator>8:live:.cid.6d6b745d306c097</initiator><target>8:live:.cid.356bbe9394363300</target></addmember>  date:  2025-02-11 06:15:56.266000
msg content:  Đã xem qua 12 Job và Aply 10 Job và 2 Job đã hết hạn  date:  2025-02-11 04:59:16.342000
msg content:  <URIObject uri="https://api.asm.skype.com/v1/objects/0-ea-d11-29ea4f93a1f1871ae7dfe5b6ac555199" url_thumbnail="https://api.asm.skype.com/v1

,USERID,DATETIME,NAME,CONTENT
34,live:minhtainth06111999,2025-02-10 07:53:22,Tai Minh,Truong_Minh_Tai - checkin - 10/2/2025 - 8h-11h
33,live:.cid.ecc600d2bf5c6cac,2025-02-10 10:50:12,botSAM,all TTS Report Task Outcome: iHugeNewRevenue &...
32,live:minhtainth06111999,2025-02-10 11:01:02,Tai Minh,Truong_Minh_Tai - checkout - 10/2/2025 - 8h - ...
31,live:minhtainth06111999,2025-02-10 11:07:32,Tai Minh,Truong_Minh_Tai - checkout - 10/2/2025 - 8h - ...
30,live:.cid.6d6b745d306c097,2025-02-10 13:23:42,Ray,17391686220148:live:.cid.6d6b745d306c0978:live...
29,live:.cid.6d6b745d306c097,2025-02-10 13:26:44,Ray,17391688041098:live:.cid.6d6b745d306c0978:live...
28,live:.cid.aaffbf40fb625b3b,2025-02-10 13:33:01,Ngoc Hoang,Hoàng Thị Hồng Ngọc - check in - 10/02/2025 - ...
27,live:.cid.54891697344fcfcc,2025-02-10 13:33:40,Ngân Lê,Lê Phan Tâm Ngân - check in - 10/2/2025 - 13h3...
26,live:.cid.a72f5f0ed4b45ab0,2025-02-10 13:34:57,Nguyễn Thị Bích Ngọc,Nguyễn Thị Bích Ngọc - check in - 10/02/2025 -...
25,live:.cid.6d6b745d306c097,2025-02-10 13:43:33,Ray,[1739169298] Nguyễn Thị Bích Ngọc: Nguyễn Thị ...
